In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
df = pd.read_csv('data/diabetic_data.csv', encoding = 'latin1',low_memory=False)

In [35]:
df.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,Caucasian,Female,[0-10),6,25,1,1,41,0,1,...,No,No,No,No,No,No,No,No,No,0
1,Caucasian,Female,[10-20),1,1,7,3,59,0,18,...,No,Up,No,No,No,No,No,Ch,Yes,1
2,AfricanAmerican,Female,[20-30),1,1,7,2,11,5,13,...,No,No,No,No,No,No,No,No,Yes,0
3,Caucasian,Male,[30-40),1,1,7,2,44,1,16,...,No,Up,No,No,No,No,No,Ch,Yes,0
4,Caucasian,Male,[40-50),1,1,7,1,51,0,8,...,No,Steady,No,No,No,No,No,Ch,Yes,0


/Users/oscar/anaconda/lib/python3.6/site-packages/pandas/core/ops.py:798: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


TypeError: invalid type comparison

In [7]:
max(df.number_diagnoses.value_counts())

NO     54864
>30    35545
<30    11357
Name: readmitted, dtype: int64


In [45]:
df = pd.read_csv('data/diabetic_data.csv', encoding = 'latin1',low_memory=False)
df.readmitted[df.readmitted == 'NO' ] = 0
df.readmitted[df.readmitted != 0 ] = 1
df.drop(['encounter_id', 'patient_nbr', 'weight', 'payer_code', 'medical_specialty'], axis=1, inplace=True)
y = df['readmitted']
X = df.drop('readmitted', axis=1)
X = pd.get_dummies(X)

0    0
1    1
2    0
3    0
4    0
Name: readmitted, dtype: object

In [38]:
df_dummies = 

,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,glimepiride-pioglitazone_No,glimepiride-pioglitazone_Steady,metformin-rosiglitazone_No,metformin-rosiglitazone_Steady,metformin-pioglitazone_No,metformin-pioglitazone_Steady,change_Ch,change_No,diabetesMed_No,diabetesMed_Yes
0,6,25,1,1,41,0,1,0,0,0,...,1,0,1,0,1,0,0,1,1,0
1,1,1,7,3,59,0,18,0,0,0,...,1,0,1,0,1,0,1,0,0,1
2,1,1,7,2,11,5,13,2,0,1,...,1,0,1,0,1,0,0,1,0,1
3,1,1,7,2,44,1,16,0,0,0,...,1,0,1,0,1,0,1,0,0,1
4,1,1,7,1,51,0,8,0,0,0,...,1,0,1,0,1,0,1,0,0,1


In [46]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=42)

numeric_headers = ["time_in_hospital", "num_lab_procedures", "num_procedures", "num_medications", "number_outpatient", "number_emergency", "number_inpatient", "number_diagnoses"]
categorical_headers = ['race',
 'gender',
 'age',
 'admission_type_id',
 'discharge_disposition_id',
 'admission_source_id',
 'diag_1',
 'diag_2',
 'diag_3',
 'max_glu_serum',
 'A1Cresult',
 'metformin',
 'repaglinide',
 'nateglinide',
 'chlorpropamide',
 'glimepiride',
 'acetohexamide',
 'glipizide',
 'glyburide',
 'tolbutamide',
 'pioglitazone',
 'rosiglitazone',
 'acarbose',
 'miglitol',
 'troglitazone',
 'tolazamide',
 'examide',
 'citoglipton',
 'insulin',
 'glyburide-metformin',
 'glipizide-metformin',
 'glimepiride-pioglitazone',
 'metformin-rosiglitazone',
 'metformin-pioglitazone',
 'change',
 'diabetesMed',
 'readmitted']
for col in numeric_headers:
    X_train[col] = X_train[col].astype(np.float)
    X_test[col] = X_test[col].astype(np.float)
    
    ss = StandardScaler()
    X_train[col] = ss.fit_transform(X_train[col].values.reshape(-1, 1))
    X_test[col] = ss.transform(X_test[col].values.reshape(-1, 1))
    
X_train.head()

,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,glimepiride-pioglitazone_No,glimepiride-pioglitazone_Steady,metformin-rosiglitazone_No,metformin-rosiglitazone_Steady,metformin-pioglitazone_No,metformin-pioglitazone_Steady,change_Ch,change_No,diabetesMed_No,diabetesMed_Yes
83900,1,1,7,-1.137221,-0.612152,-0.198155,-0.493435,0.499174,0.916206,1.073274,...,1,0,1,0,1,0,0,1,1,0
88539,1,1,7,-0.802862,-0.865589,0.975792,-1.355456,-0.289396,-0.223419,-0.502060,...,1,0,1,0,1,0,0,1,1,0
19948,1,4,7,-1.137221,1.263281,-0.785128,-0.616581,-0.289396,-0.223419,-0.502060,...,1,0,1,0,1,0,0,1,1,0
47364,3,6,1,-0.134144,-1.676587,-0.785128,-1.232310,-0.289396,-0.223419,-0.502060,...,1,0,1,0,1,0,0,1,1,0
42833,1,1,7,-0.468503,-0.510777,-0.198155,-0.000852,-0.289396,-0.223419,-0.502060,...,1,0,1,0,1,0,1,0,0,1


In [14]:
from sklearn import metrics as mt

In [15]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Input
from keras.layers import Embedding, Flatten, Merge, concatenate
from keras.models import Model

Using TensorFlow backend.


In [48]:
# This returns a tensor
inputs = Input(shape=(X_train.shape[1],))

# a layer instance is callable on a tensor, and returns a tensor
x = Dense(units=10, activation='relu')(inputs)
predictions = Dense(1,activation='sigmoid')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)

In [49]:

model.compile(optimizer='sgd',
              loss='mean_squared_error',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 2369)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                23700     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 23,711
Trainable params: 23,711
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train.values, y_train.values, epochs=10, batch_size=50, verbose=1)

from sklearn import metrics as mt
yhat = np.round(model.predict(X_test))
print(mt.confusion_matrix(y_test,yhat),mt.accuracy_score(y_test,yhat))

Epoch 1/10
76324/76324 [==============================] - 10s - loss: 0.2359 - acc: 0.5999    
Epoch 2/10
76324/76324 [==============================] - 10s - loss: 0.2297 - acc: 0.6194    
Epoch 3/10
76324/76324 [==============================] - 10s - loss: 0.2285 - acc: 0.6221    
Epoch 4/10
76324/76324 [==============================] - 10s - loss: 0.2277 - acc: 0.6235    
Epoch 5/10
76324/76324 [==============================] - 10s - loss: 0.2270 - acc: 0.6255    
Epoch 6/10
76324/76324 [==============================] - 10s - loss: 0.2266 - acc: 0.6266    
Epoch 7/10
76324/76324 [==============================] - 10s - loss: 0.2262 - acc: 0.6279    
Epoch 8/10
76324/76324 [==============================] - 10s - loss: 0.2259 - acc: 0.6295    
Epoch 9/10
76324/76324 [==============================] - 10s - loss: 0.2255 - acc: 0.6294    
Epoch 10/10
17650/76324 [=====>........................] - ETA: 8s - loss: 0.2268 - acc: 0.6267